# Image classification from scratch

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/04/27<br>
**Last modified:** 2023/11/09<br>
**Description:** Training an image classifier from scratch on the Kaggle Cats vs Dogs dataset.

## Introduction

This example shows how to do image classification from scratch, starting from JPEG
image files on disk, without leveraging pre-trained weights or a pre-made Keras
Application model. We demonstrate the workflow on the Kaggle Cats vs Dogs binary
classification dataset.

We use the `image_dataset_from_directory` utility to generate the datasets, and
we use Keras image preprocessing layers for image standardization and data augmentation.

## Setup

In [1]:
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

## Load the data: the Cats vs Dogs dataset

### Raw data download

First, let's download the 786M ZIP archive of the raw data:

In [2]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  96.5M      0  0:00:08  0:00:08 --:--:-- 69.1M


In [ ]:
!unzip -q kagglecatsanddogs_5340.zip
!ls

replace PetImages/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Now we have a `PetImages` folder which contain two subfolders, `Cat` and `Dog`. Each
subfolder contains image files for each category.

In [ ]:
!ls PetImages

### Filter out corrupted images

When working with lots of real-world image data, corrupted images are a common
occurence. Let's filter out badly-encoded images that do not feature the string "JFIF"
in their header.

In [ ]:
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} images.")

## Generate a `Dataset`

In [ ]:
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

## Visualize the data

Here are the first 9 images in the training dataset.

In [ ]:

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

## Using image data augmentation

When you don't have a large image dataset, it's a good practice to artificially
introduce sample diversity by applying random yet realistic transformations to the
training images, such as random horizontal flipping or small random rotations. This
helps expose the model to different aspects of the training data while slowing down
overfitting.

In [ ]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images


Let's visualize what the augmented samples look like, by applying `data_augmentation`
repeatedly to the first few images in the dataset:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(augmented_images[0]).astype("uint8"))
        plt.axis("off")


## Standardizing the data

Our image are already in a standard size (180x180), as they are being yielded as
contiguous `float32` batches by our dataset. However, their RGB channel values are in
the `[0, 255]` range. This is not ideal for a neural network;
in general you should seek to make your input values small. Here, we will
standardize values to be in the `[0, 1]` by using a `Rescaling` layer at the start of
our model.

## Two options to preprocess the data

There are two ways you could be using the `data_augmentation` preprocessor:

**Option 1: Make it part of the model**, like this:

```python
inputs = keras.Input(shape=input_shape)
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
...  # Rest of the model
```

With this option, your data augmentation will happen *on device*, synchronously
with the rest of the model execution, meaning that it will benefit from GPU
acceleration.

Note that data augmentation is inactive at test time, so the input samples will only be
augmented during `fit()`, not when calling `evaluate()` or `predict()`.

If you're training on GPU, this may be a good option.

**Option 2: apply it to the dataset**, so as to obtain a dataset that yields batches of
augmented images, like this:

```python
augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y))
```

With this option, your data augmentation will happen **on CPU**, asynchronously, and will
be buffered before going into the model.

If you're training on CPU, this is the better option, since it makes data augmentation
asynchronous and non-blocking.

In our case, we'll go with the second option. If you're not sure
which one to pick, this second option (asynchronous preprocessing) is always a solid choice.

## Configure the dataset for performance

Let's apply data augmentation to our training dataset,
and let's make sure to use buffered prefetching so we can yield data from disk without
having I/O becoming blocking:

In [ ]:
# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

## Build a model

We'll build a small version of the Xception network. We haven't particularly tried to
optimize the architecture; if you want to do a systematic search for the best model
configuration, consider using
[KerasTuner](https://github.com/keras-team/keras-tuner).

Note that:

- We start the model with the `data_augmentation` preprocessor, followed by a
 `Rescaling` layer.
- We include a `Dropout` layer before the final classification layer.

In [ ]:

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # We specify activation=None so as to return logits
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
pip install livelossplot


## Train the model

In [ ]:
from livelossplot import PlotLossesKeras
from tensorflow import keras

# กำหนด epochs และ callbacks
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
    PlotLossesKeras()  # เพิ่ม callback สำหรับ livelossplot
]

# Compile โมเดล
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)

# เริ่มฝึกโมเดล
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)


We get to >90% validation accuracy after training for 25 epochs on the full dataset
(in practice, you can train for 50+ epochs before validation performance starts degrading).

## Run inference on new data

Note that data augmentation and dropout are inactive at inference time.

In [ ]:
img = keras.utils.load_img("PetImages/Cat/6779.jpg", target_size=image_size)
plt.imshow(img)

img_array = keras.utils.img_to_array(img)
img_array = keras.ops.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = float(keras.ops.sigmoid(predictions[0][0]))
print(f"This image is {100 * (1 - score):.2f}% cat and {100 * score:.2f}% dog.")

สร้างmodelmake_modelX2

In [ ]:
def make_modelX2(input_shape, num_classes):
    # ฟังก์ชันสำหรับสร้างโมเดล CNN
    inputs = keras.Input(shape=input_shape)  # สร้าง input layer ที่รับข้อมูลภาพตาม input_shape

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)  # Normalize ค่า pixel ให้อยู่ในช่วง [0, 1]
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)  # Convolutional layer (128 filters, ขนาด kernel = 3x3)
    x = layers.BatchNormalization()(x)  # Normalization เพื่อปรับปรุงการเรียนรู้ของโมเดล
    x = layers.Activation("relu")(x)  # ใช้ฟังก์ชัน Activation แบบ ReLU
    x = layers.Dropout(0.3)(x)  # Dropout เพื่อป้องกัน Overfitting (อัตราการสุ่ม = 30%)

    previous_block_activation = x  # เก็บค่าจาก block นี้ไว้ใช้สำหรับ residual connection

    for size in [256, 512, 728]:  # Loop สร้าง block CNN ขนาดต่าง ๆ
        x = layers.Activation("relu")(x)  # ใช้ ReLU กับ output ก่อนเข้า convolution
        x = layers.SeparableConv2D(size, 3, padding="same")(x)  # ใช้ separable convolution เพื่อลดจำนวนพารามิเตอร์
        x = layers.BatchNormalization()(x)  # Normalization เพื่อให้ค่า output มีเสถียรภาพ

        x = layers.Activation("relu")(x)  # ใช้ ReLU อีกครั้ง
        x = layers.SeparableConv2D(size, 3, padding="same")(x)  # Separable convolution รอบที่ 2
        x = layers.BatchNormalization()(x)  # Normalization อีกครั้ง

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)  # Max Pooling เพื่อลดขนาด spatial dimensions

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(previous_block_activation)
        # Convolution 1x1 เพื่อลดหรือเพิ่ม channel ให้ตรงกับขนาดของ x
        x = layers.add([x, residual])  # บวก residual connection เข้ากับ output ของ block ปัจจุบัน
        previous_block_activation = x  # อัพเดตค่า previous_block_activation สำหรับ block ถัดไป

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)  # Convolution สุดท้าย (ขนาด 1024 filters)
    x = layers.BatchNormalization()(x)  # Normalization หลัง convolution
    x = layers.Activation("relu")(x)  # ใช้ ReLU เป็น activation function

    x = layers.GlobalAveragePooling2D()(x)  # Global Average Pooling เพื่อลด dimension เหลือเพียง 1x1

    if num_classes == 2:  # กรณี binary classification
        units = 1  # output = 1 node
    else:  # กรณี multi-class classification
        units = num_classes  # output = จำนวน class

    x = layers.Dropout(0.25)(x)  # Dropout ก่อนชั้นสุดท้าย (อัตรา 25%)
    outputs = layers.Dense(units, activation=None)(x)  # Fully connected layer สุดท้าย (activation=None สำหรับ logits)

    return keras.Model(inputs, outputs)  # Return โมเดลสำเร็จรูป


In [ ]:
from livelossplot import PlotLossesKeras
from tensorflow import keras

# กำหนด epochs และ callbacks
epochs = 15  # ตั้งค่าสูงขึ้น แต่ EarlyStopping จะหยุดเองเมื่อโมเดลไม่พัฒนา
callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras", save_best_only=True, monitor="val_loss"),  # บันทึกเฉพาะโมเดลที่ดีที่สุด
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),  # หยุดเมื่อ val_loss ไม่ลดลงใน 5 รอบ
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6),  # ลด learning rate เมื่อ val_loss ไม่พัฒนา
    PlotLossesKeras()  # แสดงกราฟ loss/accuracy ระหว่างเทรน
]

# Compile โมเดล
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),  # ใช้ Adam optimizer (learning rate เริ่มต้น 3e-4)
    loss=keras.losses.BinaryCrossentropy(from_logits=True),  # Loss สำหรับ binary classification
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],  # วัดผลด้วย accuracy
)

# เริ่มฝึกโมเดล
history = model.fit(
    train_ds,  # ข้อมูลเทรน
    epochs=epochs,  # จำนวนรอบการเทรนสูงสุด
    callbacks=callbacks,  # ใช้ callbacks ที่กำหนด
    validation_data=val_ds,  # ข้อมูล validation
    verbose=1  # แสดงผลระหว่างการฝึก
)

# บันทึกโมเดลที่ดีที่สุด
model.save("my_model_best")  # บันทึกโมเดลในรูปแบบ TensorFlow SavedModel
print("Model training complete. Best model saved successfully.")


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


In [ ]:
y_true = []  # เก็บ labels ที่แท้จริง
y_pred = []  # เก็บค่าทำนายจากโมเดล

# ทำการวนลูปเพื่อทำนายค่าใน test_ds
for images, labels in test_ds:  # test_ds เป็นชุดข้อมูล test
    predictions = model.predict(images)  # ทำนายค่าจากโมเดล
    y_true.extend(labels.numpy())  # แปลง labels เป็น numpy และเก็บใน y_true
    y_pred.extend(np.round(np.squeeze(predictions)))  # แปลงผลทำนาย logits -> binary (0/1)


In [ ]:
# คำนวณ accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# แสดง classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Cat", "Dog"]))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random


In [ ]:
def plot_predictions(images, true_labels, predicted_labels, class_names):
    plt.figure(figsize=(15, 10))
    for i in range(len(images)):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].astype("uint8"))  # แสดงภาพ
        plt.title(
            f"True: {class_names[true_labels[i]]}\nPred: {class_names[int(predicted_labels[i])]}"
        )
        plt.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
# สุ่ม 10 batch จาก test_ds
random_indices = random.sample(range(len(test_ds)), 10)
sample_images = []
sample_true_labels = []
sample_predicted_labels = []

# ดึงข้อมูลภาพจากชุด test
for i, (images, labels) in enumerate(test_ds):
    if i in random_indices:
        sample_images.extend(images.numpy())  # ดึงภาพ
        sample_true_labels.extend(labels.numpy())  # ดึง labels
        predictions = model.predict(images)  # ทำนาย
        sample_predicted_labels.extend(np.round(np.squeeze(predictions)))  # logits -> binary (0/1)
    if len(sample_images) >= 10:
        break

# จำกัดแค่ 10 รูป
sample_images = sample_images[:10]
sample_true_labels = sample_true_labels[:10]
sample_predicted_labels = sample_predicted_labels[:10]


In [ ]:
# ระบุชื่อคลาส (Cat/Dog)
class_names = ["Cat", "Dog"]

# แสดงผลการทำนาย
plot_predictions(sample_images, sample_true_labels, sample_predicted_labels, class_names)
